# Desafio Telecom X - Parte 2: Previsão de Churn com Machine Learning

## 1. Introdução

O objetivo deste projeto é construir e avaliar modelos de Machine Learning capazes de prever a evasão de clientes (churn) em uma empresa de telecomunicações. A partir dos dados tratados na Parte 1, o foco é preparar os dados para modelagem, treinar diferentes algoritmos, avaliar suas performances e, por fim, extrair insights sobre os principais fatores que influenciam a decisão de um cliente cancelar o serviço.

In [ ]:
# --- 1. Importação e Carregamento dos Dados ---
import pandas as pd
df = pd.read_csv('/dados_tratados.csv')

# --- 2. Pré-processamento ---
df = df.drop('customerID', axis=1)
df_encoded = pd.get_dummies(df, drop_first=True)

# --- 3. Separação de Features (X) e Alvo (y) ---
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

# --- 4. Divisão em Dados de Treino e Teste ---
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 5. Scaling dos Dados (NOVO PASSO!) ---
from sklearn.preprocessing import StandardScaler

# Cria o objeto scaler
scaler = StandardScaler()

# "Aprende" a escala com os dados de TREINO e os transforma
X_train_scaled = scaler.fit_transform(X_train)

# Apenas transforma os dados de TESTE com a mesma escala aprendida
X_test_scaled = scaler.transform(X_test)


print("Dados preparados, divididos e escalados com sucesso!")

Dados preparados, divididos e escalados com sucesso!


## 2. Treinamento e Avaliação dos Modelos

Nesta etapa, dois modelos de classificação foram treinados e avaliados para determinar qual possui a melhor performance na previsão de churn.

In [ ]:
# --- Modelo 1: Regressão Logística ---
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Instancia o modelo (max_iter não é mais tão crítico, mas podemos manter)
modelo_logistico = LogisticRegression(max_iter=1000)

# ATENÇÃO: Usando os dados escalados (scaled) para treinar
modelo_logistico.fit(X_train_scaled, y_train)

# E usando os dados de teste escalados para prever
previsoes_logistico = modelo_logistico.predict(X_test_scaled)

print("--- Resultados do Modelo de Regressão Logística ---")
print(f"Acurácia: {accuracy_score(y_test, previsoes_logistico) * 100:.2f}%")
print("\nRelatório de Classificação:")
print(classification_report(y_test, previsoes_logistico))

--- Resultados do Modelo de Regressão Logística ---
Acurácia: 79.60%

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.84      0.89      0.86      1033
           1       0.64      0.54      0.58       374

    accuracy                           0.80      1407
   macro avg       0.74      0.71      0.72      1407
weighted avg       0.79      0.80      0.79      1407



In [ ]:
# --- Modelo 2: Random Forest ---
from sklearn.ensemble import RandomForestClassifier

# Instancia e treina o modelo
modelo_floresta = RandomForestClassifier(random_state=42)
modelo_floresta.fit(X_train, y_train)

# Faz as previsões e avalia o modelo
previsoes_floresta = modelo_floresta.predict(X_test)

print("--- Resultados do Modelo Random Forest ---")
print(f"Acurácia: {accuracy_score(y_test, previsoes_floresta) * 100:.2f}%")
print("\nRelatório de Classificação:")
print(classification_report(y_test, previsoes_floresta))

--- Resultados do Modelo Random Forest ---
Acurácia: 77.75%

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.82      0.89      0.85      1033
           1       0.61      0.46      0.52       374

    accuracy                           0.78      1407
   macro avg       0.71      0.68      0.69      1407
weighted avg       0.76      0.78      0.77      1407



## 3. Análise do Melhor Modelo e Fatores de Influência

Após comparar os modelos, a Regressão Logística foi escolhida por ter um melhor desempenho na métrica de recall para a classe "churn". Agora, vamos investigar quais fatores (features) este modelo considerou mais importantes para tomar suas decisões.

In [ ]:
# Extrai os coeficientes do modelo de Regressão Logística
feature_names = X.columns
coeficientes = modelo_logistico.coef_[0]

# Cria um DataFrame para visualizar a importância de cada fator
importancia_features = pd.DataFrame({'Fator': feature_names, 'Peso': coeficientes})

# Ordena os fatores pela sua importância (valor absoluto do peso)
importancia_features['Importancia'] = importancia_features['Peso'].abs()
importancia_ordenada = importancia_features.sort_values(by='Importancia', ascending=False)
importancia_ordenada = importancia_ordenada.drop(columns='Importancia')

print("--- Fatores que Mais Influenciam o Churn (do maior para o menor impacto) ---")
print("Pesos positivos aumentam a chance de Churn. Pesos negativos diminuem a chance de Churn.")
print(importancia_ordenada.head(15))

--- Fatores que Mais Influenciam o Churn (do maior para o menor impacto) ---
Pesos positivos aumentam a chance de Churn. Pesos negativos diminuem a chance de Churn.
                             Fator      Peso
1                           tenure -1.504375
3                    Charges.Total  0.762711
25               Contract_Two year -0.611861
10     InternetService_Fiber optic  0.549261
2                  Charges.Monthly -0.445468
24               Contract_One year -0.297896
28  PaymentMethod_Electronic check  0.181594
26            PaperlessBilling_Yes  0.176807
9                MultipleLines_Yes  0.176481
21                 StreamingTV_Yes  0.155072
23             StreamingMovies_Yes  0.152699
13              OnlineSecurity_Yes -0.146230
19                 TechSupport_Yes -0.120072
0                    SeniorCitizen  0.087943
6                   Dependents_Yes -0.079903


## 4. Conclusão e Recomendações Estratégicas

### Resumo do Projeto
O objetivo deste projeto foi desenvolver um modelo de machine learning para prever a evasão de clientes (churn) na empresa Telecom X e identificar os principais fatores que influenciam essa decisão.

### Desempenho e Escolha do Modelo
Foram treinados e avaliados dois modelos de classificação: Regressão Logística e Random Forest. O modelo de **Regressão Logística foi escolhido como o melhor**, pois apresentou um desempenho superior na tarefa mais crítica para o negócio: identificar clientes com risco real de churn, atingindo um **recall de 54%** para a classe de churners, contra 46% do Random Forest.

### Principais Insights
A análise do modelo campeão revelou os principais fatores de risco e retenção. Os clientes com maior probabilidade de cancelar são aqueles com **internet de Fibra Óptica** e que utilizam **meios de pagamento e faturamento digitais**. Em contrapartida, o fator de maior peso para a retenção de um cliente é a adesão a **contratos de longo prazo (1 e 2 anos)**, seguido pela contratação de serviços de **suporte e segurança**.

### Recomendações Estratégicas
Com base nos insights, as seguintes ações são recomendadas:
1.  **Fidelização por Contrato:** Criar campanhas de marketing agressivas para incentivar clientes de planos mensais a migrarem para contratos de 1 ou 2 anos, oferecendo descontos ou benefícios claros.
2.  **Investigar o Serviço de Fibra:** Realizar uma análise aprofundada (ex: pesquisa de satisfação) com os clientes de Fibra Óptica para entender a causa da alta evasão. O problema está no preço, na qualidade do serviço ou na forte concorrência?
3.  **Agregar Valor com Segurança:** Promover pacotes que incluam Suporte Técnico e Segurança Online como diferenciais, uma vez que estes serviços provaram ser importantes para a retenção.